In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModelForQuestionAnswering.from_pretrained("medicalai/ClinicalBERT")

In [ ]:
# Example/dummy dataset: each sample contains a context, a question, and the ground-truth answer.
dataset = load_dataset('json', data_files='dataset.json')['train']

In [ ]:
def flatten_squad_like_data(example):
    new_contexts = []
    new_questions = []
    new_answers = []

    # Pull out the context just once
    context_text = example["context"]

    for i in range(0,  len(context_text)):
        # Each 'qa' within 'qas' is a question + list of answers
        for qa in example["qas"][i]:
            answers = qa["answers"]
            for answer in answers:
                new_contexts.append(context_text[i])
                new_questions.append(qa["question"])
                new_answers.append({"text": answer["text"], "answer_start": answer["answer_start"]})

    # Return the new, flattened lists. We rely on the 'map' call with 'batched=True'
    # to automatically collate these into a single output set of columns.
    return {
        "context": new_contexts,
        "question": new_questions,
        "answers": new_answers
    }

In [ ]:
def preprocess_function(examples):
    """
    For each example, tokenize the question and context together. Also, convert the raw answer span (start position in chars)
    into token indices that the model will use as labels.
    """
    # Tokenize question and context together.
    inputs = tokenizer(examples["question"], examples["context"],
                       truncation=True,
                       padding="max_length",
                       max_length=256,
                       return_offsets_mapping=True)

    offset_mappings = inputs.pop("offset_mapping")  # remove offsets (used only to align char indices)
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mappings):
        # For the first answer in each example
        answer = examples["answers"][i]["text"]
        answer_start_char = examples["answers"][i]["answer_start"]
        answer_end_char = answer_start_char + len(answer)

        # Find the token indices corresponding to the start and end character positions.
        start_index = None
        end_index = None
        for idx, (start_char, end_char) in enumerate(offsets):
            if start_char <= answer_start_char < end_char:
                start_index = idx
            if start_char < answer_end_char <= end_char:
                end_index = idx
        # In some cases the answer might not align perfectly with a token span.
        if start_index is None:
            start_index = 0
        if end_index is None:
            end_index = len(offsets) - 1

        start_positions.append(start_index)
        end_positions.append(end_index)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

train_dataset = dataset.map(
    flatten_squad_like_data,
    batched=True,
    remove_columns=dataset.column_names
)

# Tokenize the dataset.
tokenized_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)

# Set up training arguments.
training_args = TrainingArguments(
    output_dir="./results",
    #evaluation_strategy="no",  # for simplicity we won't run an evaluation loop here
    num_train_epochs=2,
    per_device_train_batch_size=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=1,
    save_steps=1000  # Save infrequently for the demo
)

# Initialize the Trainer with our model, training args, and tokenized dataset.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Fine tune the model on the QA dataset.
trainer.train()

# -------------------------
# Testing/inference step:
# Let’s take a question and its context, then have the model extract an answer.

question = "What medication was given?"
context = "In the hospital, the patient was administered amoxicillin for a bacterial infection."

# Tokenize the input: note that for QA we combine question and context.
inputs = tokenizer(question, context, return_tensors="pt", truncation=True, padding="max_length", max_length=256)

with torch.no_grad():
    outputs = model(**inputs)

# The model outputs 'start_logits' and 'end_logits', which represent the probability distribution
# for the start and end of the answer span.
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Identify the most likely token positions for the start and end of the answer.
start_index = torch.argmax(start_logits, dim=1).item()
end_index = torch.argmax(end_logits, dim=1).item()

# Decode the tokens corresponding to the predicted span.
input_ids = inputs["input_ids"].squeeze()
answer_tokens = input_ids[start_index:(end_index + 1)]
answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

# Output the results.
print("Question:", question)
print("Predicted Answer:", answer)

In [ ]:
# Get the contexts from the most similar questions


In [ ]:
question1 = "Que efeito tem o fármaco Rember em Alzheimer leve a moderado?"
context_CUF = """A doença de Alzheimer constitui numa alteração neurológica que causa perda de memória e declínio cognitivo progressivos. Atualmente é o tipo de demência mais comum, sendo responsável por 60% a 80% dos casos nos Estados Unidos: em 2013, 6,8 milhões de pessoas foram diagnosticadas com demência, dos quais cinco milhões classificados como Alzheimer. Em 2050, espera-se que os números dupliquem.

A doença de Alzheimer, de instalação insidiosa e progressão lenta, afeta, primeira e predominantemente, a memória episódica, com o doente a começar a ter dificuldades em lembrar-se de fragmentos recentes da sua vida (onde coloca os objetos, os recados, o que comeu no dia anterior, em que dia do mês está).

Sintomas
À perda de memória vão-se juntando lentamente outros sintomas característicos:

Dificuldade em reconhecer pessoas
Discurso mais pobre e entrecortado à procura de palavras
Orientação em espaços fica cada vez mais difícil
Com o tempo começam também a surgir as primeiras alterações do comportamento, sendo frequentes as alucinações visuais e a atividade delirante (o doente achar que o roubam ou perseguem), resultando em agitação e agressividade
Este conjunto de dificuldades aumenta até ser suficiente para a pessoa deixar de viver de forma autónoma, tendo que ser ajudada em tarefas antes realizadas de forma natural, como cozinhar, vestir-se, lavar-se, lidar com eletrodomésticos ou dinheiro.

Causas
Tal como nos outros tipos de demência, a Alzheimer trata-se de uma doença neuro degenerativa, o que significa que há perda progressiva de células cerebrais ao longo do tempo: o tecido cerebral tem cada vez menos células nervosas e vão-se perdendo as conexões entre elas.

Diagnóstico
O exame neurológico é tipicamente normal nas fases iniciais da Doença de Alzheimer, tal como os exames de imagem que, se não forem também regulares, mostram apenas atrofia dos hipocampos, formações anatómicas existentes na parte interna dos hemisférios cerebrais que têm um papel fundamental na consolidação e evocação de novas memórias. Em fases mais avançadas, os doentes desenvolvem muitas vezes sinais de Parkinson (lentidão e rigidez) e os testes mostram atrofia de todo o cérebro.

Tratamento
Não há cura conhecida para a Doença de Alzheimer, já que a morte das células cerebrais não pode ser revertida. No entanto, existem intervenções terapêuticas que melhoram a vida dos doentes e facilitam o controlo da demência, incluindo:

o controlo de outras patologias concomitantes
a terapia ocupacional
o envolvimento em grupos e serviços de apoio
a terapêutica medicamentosa: fármacos que podem reduzir os sintomas e ajudar a melhorar a qualidade de vida dos pacientes- inibidores da colinesterase (donepezilo, rivastigmina, tacrina) e memantina.
Os cuidados de apoio nas atividades diárias tornam-se mais importantes à medida que a pessoa é menos capaz de viver de forma independente.

Prevenção
Existem fatores de risco que não podem ser prevenidos como o envelhecimento, uma história familiar de Alzheimer e a genética. No entanto, há alguns fatores que podem ajudar a prevenir a doença:

a atividade física regular
a prevenção e tratamento das doenças cardiovasculares (sobretudo o controle da pressão arterial)
a prevenção e tratamento da diabetes, da obesidade e tabagismo
Em geral, uma dieta variada e saudável e permanecer mental e socialmente ativo pode reduzir o risco de doença de Alzheimer."""

# Tokenize the input: note that for QA we combine question and context.
inputs = tokenizer(question1, context_CUF, return_tensors="pt", truncation=True, padding="max_length", max_length=256)

with torch.no_grad():
    outputs = model(**inputs)

# The model outputs 'start_logits' and 'end_logits', which represent the probability distribution
# for the start and end of the answer span.
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Identify the most likely token positions for the start and end of the answer.
start_index = torch.argmax(start_logits, dim=1).item()
end_index = torch.argmax(end_logits, dim=1).item()

# Decode the tokens corresponding to the predicted span.
input_ids = inputs["input_ids"].squeeze()
answer_tokens = input_ids[start_index:(end_index + 1)]
answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

# Output the results.
print("Question:", question1)
print("Predicted Answer:", answer)

In [ ]:
# CALCULATES THE SIMILARITY FROM USER QUESTION AND DATASET QUASETIONS

import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load MedicalBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModelForQuestionAnswering.from_pretrained("medicalai/ClinicalBERT")
model.eval()  # Set the model to evaluation mode

def get_embedding(text):
    """
    Calculates the embedding for a given text using the MedicalBERT model.

    This function tokenizes the input text, runs the model to extract the hidden states,
    and then performs mean pooling over the last hidden state to generate a fixed-size embedding.
    """
    inputs = tokenizer(text, return_tensors='pt')

    with torch.no_grad():
        # Enable output_hidden_states so we can use the hidden representations
        outputs = model(**inputs, output_hidden_states=True)

    # Use the last hidden state [batch, sequence_length, hidden_dim]
    # and then perform mean pooling over the sequence_length dimension.
    last_hidden_state = outputs.hidden_states[-1]
    embedding = last_hidden_state.mean(dim=1)  # shape: [batch, hidden_dim]

    return embedding.squeeze(0)  # Remove batch dimension for similarity computation

# Sample dataset (Extend with your full dataset as needed)
dataset = [
    {
        "context": "Alzheimer's disease (AD) is a neurodegenerative disease that usually starts slowly and progressively worsens. It is the cause of 60–70% of cases of dementia. The most common early symptom is difficulty in remembering recent events.",
        "qas": [
            {
                "question": "What is Alzheimer's disease?",
                "answers": [
                    {
                        "text": "a neurodegenerative disease that usually starts slowly and progressively worsens",
                        "answer_start": 22
                    }
                ]
            },
            {
                "question": "What is the most common early symptom of Alzheimer's?",
                "answers": [
                    {
                        "text": "difficulty in remembering recent events",
                        "answer_start": 154
                    }
                ]
            }
        ]
    },
    {
        "context": "Alzheimer's disease is the most common cause of dementia — a continuous decline in thinking, behavioral and social skills that affects a person's ability to function independently.",
        "qas": [
            {
                "question": "What does Alzheimer's disease cause?",
                "answers": [
                    {
                        "text": "a continuous decline in thinking, behavioral and social skills",
                        "answer_start": 53
                    }
                ]
            },
            {
                "question": "How does dementia affect a person?",
                "answers": [
                    {
                        "text": "affects a person's ability to function independently",
                        "answer_start": 117
                    }
                ]
            }
        ]
    },
    # ... add more entries as needed
]

# User-provided question
user_question = "What are the symptoms of dementia?"

# Calculate the embedding for the user question
user_embedding = get_embedding(user_question)

# Compute cosine similarity with each question in the dataset and store results
results = []
for entry in dataset:
    context = entry["context"]
    for qa in entry["qas"]:
        question = qa["question"]
        question_embedding = get_embedding(question)
        # Compute cosine similarity between the user question embedding and the dataset question embedding
        similarity = F.cosine_similarity(user_embedding, question_embedding, dim=0)
        results.append({
            "context": context,
            "question": question,
            "similarity": similarity.item()
        })

# Get the top 3 similar questions (highest cosine similarity)
top_results = sorted(results, key=lambda x: x["similarity"], reverse=True)[:3]

# Display the top matching contexts, questions, and similarity scores
print("Top 3 similar questions and their contexts:")
for result in top_results:
    print(f"Similarity: {result['similarity']:.4f}")
    print(f"Question: {result['question']}")
    print(f"Context: {result['context']}\n")
